<a href="https://colab.research.google.com/github/yisi22/Real-Estate-Investment/blob/main/Data_Center.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

from pandas_datareader import data as pdr

import datetime as dt
import yfinance as yf
import seaborn as sns
import random
yf.pdr_override()

In [2]:
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.cluster.hierarchy import fcluster

from sklearn.preprocessing import StandardScaler

from sklearn import cluster, covariance, manifold
from sklearn.metrics import adjusted_mutual_info_score

from sklearn.cluster import AgglomerativeClustering

import scipy.cluster.hierarchy as sch

from sklearn.cluster import KMeans, AgglomerativeClustering,AffinityPropagation, DBSCAN
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram, linkage, cophenet
from scipy.spatial.distance import pdist
from sklearn.metrics import adjusted_mutual_info_score
from sklearn import cluster, covariance, manifold
import matplotlib.ticker as ticker
from itertools import cycle
from sklearn import metrics
from statsmodels.tsa.stattools import coint

In [ ]:
ticker = ['EQIX', 'DLR', 'DBRG', 'AJBU.SI', 'AMT','IRM','BX','KKR','COR','GDS','0788.HK', 'NXT.AX']
data = yf.download(ticker, start = '2010-1-1')
data_ticker = data['Adj Close']

In [ ]:
data_ticker

In [ ]:
# check missing value
def check_missing_value (data):
    missing_values = data.isnull().mean().sort_values(ascending = False)
    missing_value_frame = missing_values.to_frame().head(20).style.format('{:,.3%}'.format)
    drop_list = sorted(list(missing_values[missing_values > 0.3].index))
    wrangled_data = data.drop(columns = drop_list, axis =1)
    wrangled_data = wrangled_data.ffill()
    wrangled_data = wrangled_data.dropna(axis =0)
    return missing_value_frame, wrangled_data

In [ ]:
# get the wrangled data
missing_value_frame, wrangled_data = check_missing_value(data_ticker)

## Clustering using AGNES

In [ ]:
# Use AGNES to Cluster
def wrangle_data(data):
    missing_fractions = data.isnull().mean().sort_values(ascending = False)
    drop_list = sorted(list(missing_fractions[missing_fractions > 0.5].index))
    data = data.drop(labels = drop_list, axis =1)
    data = data.ffill()
    returns = data.pct_change().mean()*252
    returns = pd.DataFrame(returns, columns = pd.Index(['Returns']))
    returns['Volatility'] = data.pct_change().std() * np.sqrt(252)
    data_test = np.asarray([np.asarray(returns['Returns']), np.asarray(returns['Volatility'])]).T
    scaler = StandardScaler().fit(data_test)
    rescaledDataset = pd.DataFrame(scaler.fit_transform(data_test), columns = returns.columns, index = returns.index)
    return rescaledDataset
def linkage_dendrogram(X, distance):
    Z = linkage(X, method = 'ward')
    plt.figure(figsize = [18,10])
    plt.title('Stock Dendograms')
    dendrogram(Z, labels = X.index)
    plt.show()
    distance_threshold = distance
    clusters = fcluster(Z, distance_threshold, criterion = 'distance')
    chosen_clusters = pd.DataFrame(data = clusters, columns = ['cluster'])
    return chosen_clusters
def clustering(X, nclust):
    hc = AgglomerativeClustering(n_clusters = nclust, affinity = 'euclidean', linkage = 'ward')
    clust_labels = hc.fit_predict(X)
    fig = plt.figure(figsize = (16,10))
    ax = fig.add_subplot(111)
    scatter = ax.scatter(X.iloc[:, 0], X.iloc[:, 1], c=clust_labels, cmap = 'rainbow')
    ax.set_title('Hierachial')
    ax.set_xlabel('Average Return')
    ax.set_ylabel('Volitility')
    plt.colorbar(scatter)
    return hc, clust_labels

In [ ]:
rescaledDataset = wrangle_data(data_ticker)

In [ ]:
rescaledDataset

In [ ]:
chosen_clusters = linkage_dendrogram(rescaledDataset, 2)

In [ ]:
hc, clust_labels = clustering(rescaledDataset, 5)

In [ ]:
cluster_df = pd.DataFrame(rescaledDataset.index, index = clust_labels, columns = ['Ticker'])

In [ ]:
cluster_df = cluster_df.sort_index()

In [ ]:
cluster_df.loc[0]

## K-Means Clustering

In [ ]:
def plot_SquaredError(X, max_loop):
    distorsions = []
    for k in range(2, max_loop):
        kmeans = KMeans(n_clusters = k)
        kmeans.fit(X)
        distorsions.append(kmeans.inertia_)
    fig = plt.figure(figsize=(16, 8))
    plt.plot(range(2, max_loop), distorsions)
    plt.xticks([i for i in range(2, max_loop)], rotation=75)
    plt.grid(True)

def plot_SilhouetteScore(X, max_loop):
    silhouette_score = []
    for k in range(2, max_loop):
        kmeans = KMeans(n_clusters = k,
                        random_state = 10,
                        n_init = 10)
        kmeans.fit(X)
        silhouette_score.append(metrics.silhouette_score(X, kmeans.labels_, random_state=10))
    fig = plt.figure(figsize=(16, 10))
    plt.plot(range(2, max_loop), silhouette_score)
    plt.xticks([i for i in range(2, max_loop)], rotation=75)
    plt.grid(True)

In [ ]:
plot_SquaredError(rescaledDataset, 3)

In [ ]:
plot_SilhouetteScore(rescaledDataset, 3)

In [ ]:
def K_means(X, nclust):
    k_means = cluster.KMeans(n_clusters=nclust)
    k_means.fit(X)
    target_labels = k_means.predict(X)
    return k_means, target_labels

def plot_Kmeans_results(X, k_means):
    centroids = k_means.cluster_centers_
    fig = plt.figure(figsize=(16,10))
    ax = fig.add_subplot(111)
    scatter =(ax.scatter(X.iloc[ : ,0],
             X.iloc[ : ,1],
             c = k_means.labels_,
             cmap = "rainbow",
             label = X.index))
    ax.set_title("k-Means results")
    ax.set_xlabel("Average Return")
    ax.set_ylabel("Volatility")
    plt.colorbar(scatter)
    plt.plot(centroids[:,0],
         centroids[:,1],
         "sg",
         markersize = 15,
         color = "black")

In [ ]:
k_means, target_labels = K_means(rescaledDataset,4)

In [ ]:
plot_Kmeans_results(rescaledDataset, k_means)

In [ ]:
clustered_series =pd.DataFrame(data = rescaledDataset.index, index = k_means.labels_.flatten(), columns = ['Ticker'])

In [ ]:
clustered_series = clustered_series.sort_index()

In [ ]:
len(clustered_series.loc[2])

In [ ]:
def plot_ClusterMemberCounts(X, k_means):
    # show number of stocks in each cluster
    clustered_series =(pd.Series(index = X.index, data = k_means.labels_.flatten()))

    # clustered stock with its cluster label
    clustered_series_all = pd.Series(index=X.index, data=k_means.labels_.flatten())
    clustered_series = clustered_series[clustered_series != -1]

    #plot the counts
    plt.figure(figsize=(16,8))
    counts =clustered_series.value_counts().sort_index()
    plt.barh(counts.index,counts)
    plt.title("Cluster Member Counts")
    plt.xlabel("Stocks in Cluster")
    plt.ylabel("Cluster Number")

    plt.show()

In [ ]:
plot_ClusterMemberCounts(rescaledDataset, k_means)

## Affinity Propagation

In [ ]:
def Affinity_Propagation(X):
    ap = AffinityPropagation()
    ap.fit(X)
    clust_labels_AP = ap.predict(X)
    return ap,clust_labels_AP

def plot_affinity(X, crust_labels_AP):
    fig = plt.figure(figsize=(16,10)
                )
    ax = fig.add_subplot(111)

    scatter = ax.scatter(X.iloc[:,0],
                     X.iloc[:,1],
                     c = crust_labels_AP,
                     cmap = "rainbow")

    ax.set_title("Affinity")
    ax.set_xlabel("Average Return")
    ax.set_ylabel("Volatility")

    plt.colorbar(scatter)

In [ ]:
ap,clust_labels_AP = Affinity_Propagation(rescaledDataset)

In [ ]:
plot_affinity(rescaledDataset, clust_labels_AP)

In [ ]:
def stock_in_each_cluster(X,ap):
    # show number of stocks in each cluster
    clustered_series_ap = pd.Series(index=X.index, data=ap.labels_.flatten())
    clustered_df_ap = pd.DataFrame(data=X.index, index=ap.labels_.flatten(), columns = ['Ticker'])
    clustered_df_ap = clustered_df_ap.sort_index()
    # clustered stock with its cluster label
    clustered_series_all_ap = pd.Series(index=X.index, data=ap.labels_.flatten())
    clustered_series_ap = clustered_series_ap[clustered_series_ap != -1]

    return clustered_series_all_ap, clustered_series_ap, clustered_df_ap

In [ ]:
def plot_cluster_member_count(clustered_series_ap):
    plt.figure(figsize=(16,10)
          )

    plt.barh(range(len(clustered_series_ap.value_counts()
             )
         ), # cluster labels, y axis
    clustered_series_ap.value_counts())

    plt.title("Cluster Member Counts")
    plt.xlabel("Stocks in Cluster")
    plt.ylabel("Cluster Number")

    plt.show()

In [ ]:
clustered_series_all_ap, clustered_series_ap,clustered_df_ap = stock_in_each_cluster(rescaledDataset,ap)

In [ ]:
clustered_df_ap.loc[0]

In [ ]:
plot_cluster_member_count(clustered_series_ap)

## Evaluation of Different Clustering Algorithm  

In [ ]:
print("km", metrics.silhouette_score(rescaledDataset, k_means.labels_, metric='euclidean'))

print("hc", metrics.silhouette_score(rescaledDataset, hc.fit_predict(rescaledDataset), metric='euclidean'))

print("ap", metrics.silhouette_score(rescaledDataset, ap.labels_, metric='euclidean'))